In [167]:
import pandas as pd
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")

In [168]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


In [169]:
def pluralize_gift(gift):
  """
  Returns plural of a noun

  Parameters
  ----------
  gift: str
    A noun

  Return
  ------
  str
    Plural version
  """

  if gift == "goose":
    gift = gift.replace("oo", "ee")
  elif gift.endswith("y"):
    gift = gift.replace("y", "ies")
  elif gift == "partridge":
    gift = gift
  else:
    gift += 's'

  return gift

In [170]:
# Should return plural gifts
xmas['Gift.Item'].apply(pluralize_gift)

0     partridge
1         doves
2          hens
3         birds
4         rings
5         geese
6         swans
7         maids
8        ladies
9         lords
10       pipers
11     drummers
Name: Gift.Item, dtype: object

In [171]:
xmas

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN
5,6,sixth,goose,a-laying,NaN,NaN
6,7,seventh,swan,a-swimming,NaN,NaN
7,8,eighth,maid,a-milking,NaN,NaN
8,9,ninth,lady,dancing,NaN,NaN
9,10,tenth,lord,a-leaping,NaN,NaN


In [172]:
# Make a column of words we need for numeric values
cardinal = {
    "first": "one",
    "second": "two",
    "third": "three",
    "fourth": "four",
    "fifth": "five",
    "sixth": "six",
    "seventh": "seven",
    "eighth": "eight",
    "ninth": "nine",
    "tenth": "ten",
    "eleventh": "eleven",
    "twelfth": "twelve"}

In [173]:
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  Returns complete phrase

  Parameters
  ----------
  gift: str
    A noun
  num: int
    day number
  num_word: str
    word for day number
  item: str
    the gift for the day
  verb: str
    The action for the gift
  adjective: str
    Description for the gift if applicable
  location: str
    Location of gift if applicable


  Return phrase
  ------
  str
    Complete phrase as a string
  """

  # Step 1: Replace NAs with blank strings
  verb = verb if not pd.isna(verb) else ""
  adjective = adjective if not pd.isna(adjective) else ""
  location = location if not pd.isna(location) else ""

  # Step 2: If the day number is larger than 1, the gift items need pluralized!
  if num > 1:
    item = pluralize_gift(item)

  # Step 3: Figure out if a gift item starts with a vowel
  vowel = item[0].lower() in "aeiou"

  # Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  cardinal_num_word = cardinal.get(num_word, num_word)
  if num == 1:
    prefix = "a" if not vowel else "an"
  else:
    prefix = cardinal_num_word

  # Step 5: Put all of the pieces together into one string and return!
  phrase_pieces = [prefix, adjective, item, verb, location]
  phrase = " ".join(filter(None, phrase_pieces))

  return phrase.strip()

In [174]:
test_phrase1 = make_phrase(
    num=1,
    num_word="first",
    item="partridge",
    verb="",
    adjective="",
    location="in a pear tree"
)
test_phrase1

'a partridge in a pear tree'

In [175]:
test_phrase9 = make_phrase(
    9,
    "ninth",
    "lady",
    "dancing",
    "",
    ""
)
test_phrase9

'nine ladies dancing'

In [176]:
# Use the function to make a new column of the xmas column called Full.Phrase containing the sentences for the new gift on that day
xmas['Full.Phrase'] = xmas.apply(lambda row: make_phrase(row["Day"], row["Day.in.Words"], row["Gift.Item"], row["Verb"], row["Adjective"], row["Location"]), axis=1)
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location,Full.Phrase
0,1,first,partridge,NaN,NaN,in a pear tree,a partridge in a pear tree
1,2,second,dove,NaN,turtle,NaN,two turtle doves
2,3,third,hen,NaN,french,NaN,three french hens
3,4,fourth,bird,NaN,calling,NaN,four calling birds
4,5,fifth,ring,NaN,golden,NaN,five golden rings


In [177]:
def sing_day(dataset, num, phrase_col):
  """
  Returns the song verse from a selected day

  Parameters
  ----------
  dataset: dataframe
    The dataset with the song parts contained in it
  num: int
    day number
  phrase_col: str
    Name of the column to be returned

  Return song
  ------
  str
    the song verse from a selected day
  """

  # Step 1: Setup the intro line
  # convert "1" to "first" etc.
  word = {
        1: "first",
        2: "second",
        3: "third",
        4: "fourth",
        5: "fifth",
        6: "sixth",
        7: "seventh",
        8: "eighth",
        9: "ninth",
        10: "tenth",
        11: "eleventh",
        12: "twelfth"
  }
  num_word = word[num]

  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"

  # Step 2: Sing the gift phrases
  gifts = []
  for i in range(num, 0, -1):
    gifts_phrase = dataset.loc[i-1, phrase_col]
    if i == 1 and num > 1:
      gifts_phrase = "and " + gifts_phrase
    gifts.append(gifts_phrase)

  # add commas to all but last line
  comma_gifts = [gift + ',' for gift in gifts[:-1]] + [gifts[-1]]

    # Step 3: Put it all together and return
  gifts = "\n" + "\n".join(comma_gifts) + '.'
  song = intro + gifts

  return song

In [178]:
print(sing_day(xmas, 3, "Full.Phrase"))

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.


Run appropriate code to output the lyrics for the entire 12 Days of Christmas song.

In [180]:
def full_song(dataset, phrase_col):
    """
    Print all song verses for the given dataset and phrase column.

    Parameters
    ----------
    dataset: dataframe
        The dataset with the song parts contained in it
    phrase_col: str
        Name of the column to be returned
    """

    song_verses = []

    for day in range(1, 13):
        verse = sing_day(dataset, day, phrase_col)
        song_verses.append(verse)
        song_verses.append('\n\n')

    print("".join(song_verses))

In [181]:
full_song(xmas, "Full.Phrase")

On the first day of Christmas, my true love sent to me:
a partridge in a pear tree.

On the second day of Christmas, my true love sent to me:
two turtle doves,
and a partridge in a pear tree.

On the third day of Christmas, my true love sent to me:
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fourth day of Christmas, my true love sent to me:
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the fifth day of Christmas, my true love sent to me:
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the sixth day of Christmas, my true love sent to me:
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.

On the seventh day of Christmas, my true love sent to me:
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a 

In [182]:
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas2

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,email,NaN,NaN,from Cal Poly
1,2,second,point,NaN,meal,NaN
2,3,third,pen,NaN,lost,NaN
3,4,fourth,review,NaN,course,NaN
4,5,fifth,exam,NaN,practice,NaN
5,6,sixth,grader,grading,NaN,NaN
6,7,seventh,senior,stressing,NaN,NaN
7,8,eighth,mom,a-calling,NaN,NaN
8,9,ninth,party,bumping,NaN,NaN
9,10,tenth,load,of laundry,NaN,NaN


In [183]:
xmas2['Full.Phrase'] = xmas2.apply(lambda row: make_phrase(row["Day"], row["Day.in.Words"], row["Gift.Item"], row["Verb"], row["Adjective"], row["Location"]), axis=1)

In [184]:
full_song(xmas2, "Full.Phrase")

On the first day of Christmas, my true love sent to me:
an email from Cal Poly.

On the second day of Christmas, my true love sent to me:
two meal points,
and an email from Cal Poly.

On the third day of Christmas, my true love sent to me:
three lost pens,
two meal points,
and an email from Cal Poly.

On the fourth day of Christmas, my true love sent to me:
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the fifth day of Christmas, my true love sent to me:
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the sixth day of Christmas, my true love sent to me:
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On the seventh day of Christmas, my true love sent to me:
seven seniors stressing,
six graders grading,
five practice exams,
four course reviews,
three lost pens,
two meal points,
and an email from Cal Poly.

On t